In [2]:
import numpy as np
import pandas as pd
import time
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

### Extract All The Links

In [99]:
PATH = "C:\Program Files (x86)\web_driver\chromedriver.exe"
driver = webdriver.Chrome(PATH)

driver.get('https://dialnet.unirioja.es/tesis/fechalectura')

In [ ]:
%%time
absLinks=[]
max_pages = 0
while True:
    link1 = driver.find_elements_by_css_selector("span.titulo")
    for tags in link1:
        link2 = tags.find_elements_by_tag_name("a")
        for link3 in link2:
            absLinks.append(link3.get_attribute("href"))
    
    # extract links
    try:
        time.sleep(7)
        driver.find_element_by_xpath('//*[@id="pieDeListadoDeBusquedaDeAutores"]/ul/li[4]/a').click() 
        max_pages += 1
        if max_pages == 100:
            break
    # stop if no more pages available
    except NoSuchElementException:
        break

### Get Contents From Each Link

In [ ]:
data = []
for link in absLinks:
    time.sleep(5)
    driver.get(link)

    try:
        contents = driver.find_elements_by_xpath('//*[@id="resumen"]')
        for content in contents:
            language = content.text

    except NoSuchElementException:
        break

    #collect the data
    lang= {'content': language}
    data.append(lang)

In [130]:
en_sp = pd.DataFrame(data)

In [131]:
len(en_sp)

159

### Data Cleaning

In [132]:
# remove \n 
en_sp['content'] = en_sp['content'].apply(lambda x: re.sub(r'\n', '', x))

# return only rows with 'Espanol' in the text (because it contains english translate too)
en_sp['Spanish'] = en_sp['content'].apply(lambda x: x if 'Español' in x else 0)

# remove rows with 0
en_sp = en_sp[en_sp['Spanish'] != 0]

# create two new variables: Spanish_text, English_text to separate the translations
en_sp['Spanish_text'] = en_sp['Spanish'].apply(lambda x: x.split('English')[0])
en_sp['English_text'] = en_sp['Spanish'].apply(lambda x: x.split('English')[-1])

# remove 'Galego' translation in the English_text variable
en_sp['English_text'] = en_sp['English_text'].apply(lambda x: x.split('Galego')[0] if 'Galego' in x else x)

en_sp['Spanish_text'] = en_sp['Spanish_text'].apply(lambda x: x.split('Español')[-1])

In [133]:
# make sure Spanish_text and English_text do not have the same value
en_sp = en_sp[en_sp['Spanish_text'] != en_sp['English_text']]
len(en_sp)

55

In [134]:
# format dataframe
en_sp.reset_index(inplace=True)
en_sp.drop(['index', 'content', 'Spanish'], axis=1, inplace=True)

In [156]:
en_sp.head()

,Spanish_text,English_text
0,Esta tesis quiere profundizar sobre el caso de...,This thesis wants to delve into the case of th...
1,La violencia en sus diversas variantes está pr...,Violence in its several variations is present ...
2,"Esta tesis doctoral es un ensayo clínico que, ...",This doctoral thesis is a clinical trial that ...
3,La frustración es un fenómeno que puede mitiga...,Frustration can be mitigated if there has been...
4,Las políticas lingüísticas europeas promueven ...,European language policies promote the trainin...


In [157]:
en_sp['Spanish_text'][30]

'La presente tesis doctoral tiene como objetivos principales examinar los vínculos entre la trata de seres humanos y la corrupción, y analizar críticamente la respuesta del Derecho Penal frente a la participación de funcionarios públicos en la trata. Para ello, se aborda, en primer lugar, la evolución de la normativa internacional en la materia, el concepto de trata de personas y el de corrupción. De este modo, se delimita con claridad qué se entiende, en el plano teórico, por conductas corruptas relacionadas con la trata. Posteriormente, se aporta una descripción criminológica del fenómeno de la corrupción relacionada con la trata en Europa. Esto permite entender mejor quién, cómo y cuándo puede llevar a cabo estas conductas. Partiendo de esta información, se estudia la respuesta penal frente a la corrupción relacionada con la trata en España, a través del análisis dogmático del el artículo 177 bis 5 del Código penal y la jurisprudencia en la materia. Finalmente, con toda la informaci

In [158]:
en_sp['English_text'][30]

'This doctoral thesis has two main objectives: to examine the links between human trafficking and corruption, and to analyse the Criminal Law response to public officials’ involvement in trafficking. To do so, it first addresses the evolution of international regulations related to this topic, as well as the concepts of human trafficking and corruption. Thus, the concept of trafficking-related corruption is clearly delimited on a theoretical level. Subsequently, this thesis offers a criminological description of human trafficking-related corruption practices in Europe. This allows for a better understanding of how, when and by whom these conducts can be carried out. In the light of this data, the Criminal Law response to trafficking-related corruption in Spain is critically analysed, focussing on the scope of application of article 177 bis 5 of the Spanish Criminal Code and case-law on this topic. Finally, with all the information obtained, some measures to improve the investigation, p

In [159]:
en_sp.to_csv('lang_doc.csv', index=False)